# Project Notebook: Optimizing DataFrames and Processing in Chunks

## 1. Introduction 

In this project, we'll practice working with chunked dataframes and optimizing a dataframe's memory usage. We'll be working with financial lending data from Lending Club, a marketplace for personal loans that matches borrowers with investors. You can read more about the marketplace on its website.

The Lending Club's website lists approved loans. Qualified investors can view the borrower's credit score, the purpose of the loan, and other details in the loan applications. Once a lender is ready to back a loan, it selects the amount of money it wants to fund. When the loan amount the borrower requested is fully funded, the borrower receives the money, minus the origination fee that Lending Club charges.

We'll be working with a dataset of loans approved from 2007-2011 (https://bit.ly/3H2XVgC). We've already removed the desc column for you to make our system run more quickly.

If we read in the entire data set, it will consume about 67 megabytes of memory. Let's imagine that we only have 10 megabytes of memory available throughout this project, so you can practice the concepts you learned in the last two lessons.

**Tasks**

1. Read in the first five lines from `loans_2007.csv` (https://bit.ly/3H2XVgC) and look for any data quality issues.

2. Read in the first 1000 rows from the data set, and calculate the total memory usage for these rows. Increase or decrease the number of rows to converge on a memory usage under five megabytes (to stay on the conservative side).

In [1]:
# Importing pandas
import pandas as pd
pd.options.display.max_columns = 99

# Download the loans_2007.csv
!wget -O loans_2007.csv https://bit.ly/3H2XVgC


--2023-01-27 14:04:32--  https://bit.ly/3H2XVgC
Resolving bit.ly (bit.ly)... 67.199.248.11, 67.199.248.10
Connecting to bit.ly (bit.ly)|67.199.248.11|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://archive.org/download/loans_2007/loans_2007.csv [following]
--2023-01-27 14:04:32--  https://archive.org/download/loans_2007/loans_2007.csv
Resolving archive.org (archive.org)... 207.241.224.2
Connecting to archive.org (archive.org)|207.241.224.2|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://ia902300.us.archive.org/3/items/loans_2007/loans_2007.csv [following]
--2023-01-27 14:04:33--  https://ia902300.us.archive.org/3/items/loans_2007/loans_2007.csv
Resolving ia902300.us.archive.org (ia902300.us.archive.org)... 207.241.228.50
Connecting to ia902300.us.archive.org (ia902300.us.archive.org)|207.241.228.50|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 15638149 (15M) [text/csv]


In [2]:
#Read first five lines and look for data quality issues
df_loans = pd.read_csv('loans_2007.csv', nrows=5)
print(df_loans)

        id  member_id  loan_amnt  funded_amnt  funded_amnt_inv        term  \
0  1077501  1296599.0     5000.0       5000.0           4975.0   36 months   
1  1077430  1314167.0     2500.0       2500.0           2500.0   60 months   
2  1077175  1313524.0     2400.0       2400.0           2400.0   36 months   
3  1076863  1277178.0    10000.0      10000.0          10000.0   36 months   
4  1075358  1311748.0     3000.0       3000.0           3000.0   60 months   

  int_rate  installment grade sub_grade                 emp_title emp_length  \
0   10.65%       162.87     B        B2                       NaN  10+ years   
1   15.27%        59.83     C        C4                     Ryder   < 1 year   
2   15.96%        84.33     C        C5                       NaN  10+ years   
3   13.49%       339.31     C        C1       AIR RESOURCES BOARD  10+ years   
4   12.69%        67.79     B        B5  University Medical Group     1 year   

  home_ownership  annual_inc verification_status  

In [3]:
# By trying out different values, we see that 10,000 rows takes up 4MB, which is
# less than 5MB as desired
df_loans = pd.read_csv('loans_2007.csv', nrows=10000)
print(df_loans.info(memory_usage=True))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 52 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   id                          10000 non-null  int64  
 1   member_id                   10000 non-null  float64
 2   loan_amnt                   10000 non-null  float64
 3   funded_amnt                 10000 non-null  float64
 4   funded_amnt_inv             10000 non-null  float64
 5   term                        10000 non-null  object 
 6   int_rate                    10000 non-null  object 
 7   installment                 10000 non-null  float64
 8   grade                       10000 non-null  object 
 9   sub_grade                   10000 non-null  object 
 10  emp_title                   9348 non-null   object 
 11  emp_length                  9645 non-null   object 
 12  home_ownership              10000 non-null  object 
 13  annual_inc                  1000

## 2. Exploring the Data in Chunks

Let's familiarize ourselves with the columns to see which ones we can optimize. In the first lesson, we explored column types by reading in the full dataframe. In this project, let's try to understand the column types better while using dataframe chunks.

**Tasks**

For each chunk:
* How many columns have a numeric type? 
* How many have a string type?
* How many unique values are there in each string column? How many of the string columns contain values that are less than 50% unique?
* Which float columns have no missing values and could be candidates for conversion to the integer type?
* Calculate the total memory usage across all of the chunks.

In [4]:
# import function to tell whether dtype is numeric
from pandas.api.types import is_numeric_dtype


In [5]:
# No. of Columns with numeric and non-numeric dtype

df_loans_iter = pd.read_csv('loans_2007.csv', chunksize=10000)
# Use next() to get the first chunk, since df_loans is an iterable
df_loans = next(df_loans_iter)

num_numeric = 0
num_non_numeric = 0
for col in df_loans:
    if is_numeric_dtype(df_loans[col]):
        num_numeric += 1
    else:
        num_non_numeric += 1
print('Number of numeric columns: ', num_numeric)
print('Number of non-numeric columns i.e. strings: ', num_non_numeric)

Number of numeric columns:  31
Number of non-numeric columns i.e. strings:  21


In [6]:
# Number of unique columns in each string column
# NB: df_loans is currently the first chunk

df_loans_string = df_loans.select_dtypes(include=['object'])
for col in df_loans_string:
    num_unique = len(df_loans_string[col].value_counts())
    print(col, num_unique, ', Ratio: ', num_unique / 10000) # 10000 is the chunksize

# By inspecting the output below, ALL the string columns except emp_title have 
# less than 50% unique values.

term 2 , Ratio:  0.0002
int_rate 70 , Ratio:  0.007
grade 7 , Ratio:  0.0007
sub_grade 35 , Ratio:  0.0035
emp_title 8171 , Ratio:  0.8171
emp_length 11 , Ratio:  0.0011
home_ownership 3 , Ratio:  0.0003
verification_status 3 , Ratio:  0.0003
issue_d 5 , Ratio:  0.0005
loan_status 6 , Ratio:  0.0006
pymnt_plan 1 , Ratio:  0.0001
purpose 13 , Ratio:  0.0013
title 4084 , Ratio:  0.4084
zip_code 720 , Ratio:  0.072
addr_state 45 , Ratio:  0.0045
earliest_cr_line 465 , Ratio:  0.0465
revol_util 1026 , Ratio:  0.1026
initial_list_status 1 , Ratio:  0.0001
last_pymnt_d 58 , Ratio:  0.0058
last_credit_pull_d 59 , Ratio:  0.0059
application_type 1 , Ratio:  0.0001


In [7]:
# float columns without missing values
df_loans_numeric = df_loans.select_dtypes(include=['float16', 'float32', 'float64'])
for col in df_loans_numeric:
    num_missing_vals = df_loans_numeric[col].isnull().sum()
    print(col,  '=> Missing values: ', num_missing_vals) # 10000 is the chunksize

# From inspecting the output below, non of the float columns have missing values
# They are therefore all candidates for conversion to integers

member_id => Missing values:  0
loan_amnt => Missing values:  0
funded_amnt => Missing values:  0
funded_amnt_inv => Missing values:  0
installment => Missing values:  0
annual_inc => Missing values:  0
dti => Missing values:  0
delinq_2yrs => Missing values:  0
inq_last_6mths => Missing values:  0
open_acc => Missing values:  0
pub_rec => Missing values:  0
revol_bal => Missing values:  0
total_acc => Missing values:  0
out_prncp => Missing values:  0
out_prncp_inv => Missing values:  0
total_pymnt => Missing values:  0
total_pymnt_inv => Missing values:  0
total_rec_prncp => Missing values:  0
total_rec_int => Missing values:  0
total_rec_late_fee => Missing values:  0
recoveries => Missing values:  0
collection_recovery_fee => Missing values:  0
last_pymnt_amnt => Missing values:  0
collections_12_mths_ex_med => Missing values:  0
policy_code => Missing values:  0
acc_now_delinq => Missing values:  0
chargeoff_within_12_mths => Missing values:  0
delinq_amnt => Missing values:  0
pu

In [8]:
# Memory usage across all the chunks

# Reset the iterable
df_loans_iter = pd.read_csv('loans_2007.csv', chunksize=10000)
# Iterate over all the chunks
memory_used = 0
for df_loans in df_loans_iter:
    memory_used += df_loans.memory_usage(deep=True).sum()

print('Total memory across chunks: ', memory_used)

# From the output below, memory across chunks is 68.9MB

Total memory across chunks:  68905389


## 3. Optimizing String Columns

We can achieve the greatest memory improvements by converting the string columns to a numeric type. Let's convert all of the columns where the values are less than 50% unique to the category type, and the columns that contain numeric values to the `float` type.

While working with dataframe chunks:
* Determine which string columns you can convert to a numeric type if you clean them. For example, the `int_rate` column is only a string because of the % sign at the end.
* Determine which columns have a few unique values and convert them to the category type. For example, you may want to convert the grade and `sub_grade` columns.
Based on your conclusions, perform the necessary type changes across all chunks. * Calculate the total memory footprint, and compare it with the previous one.

**Clean the string columns**

Such cleaning involves e.g. stripping the '%' sign at the end of a string if doing so will make it convertible to a float.

In [9]:
df_loans_string = df_loans.select_dtypes(include=['object'])

def strip_strings(df, to_strip):
    '''
    parameters:
    - df: dataframe
    - to_strip: the set of characters to be stripped
    '''
    for col in df:
        # Ensure the column is a string. This enables the fn to work even if some
        # columns of the df are not strings.
        if not is_numeric_dtype(df[col]):
            df[col] = df[col].str.rstrip(to_strip)

# Test the strip_strings() function
strip_strings(df_loans_string, '%')
print(df_loans_string)

# In the output of this cell (below), notict that '%' has been stripped off every
# element in the 'int_rate' column.

# The above works with the assumption that '%' at the end of a string ONLY signifies percentage
# Other scenarios could arise in which '%' means something else, in which case more validation would
# need to be done.


                                                    id        term int_rate  \
40000                                           598513   36 months    14.84   
40001                                           585666   36 months    14.72   
40002                                           598114   36 months    15.21   
40003                                           598033   60 months    16.32   
40004                                           597824   36 months    16.45   
...                                                ...         ...      ...   
42533                                            72176   36 months     9.33   
42534                                            71623   36 months     8.38   
42535                                            70686   36 months     7.75   
42536  Total amount funded in policy code 1: 471701350         NaN      NaN   
42537          Total amount funded in policy code 2: 0         NaN      NaN   

      grade sub_grade                    emp_title 

<ipython-input-9-87a4a089bff1>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = df[col].str.rstrip(to_strip)


Unique values in each string column
If these are less than 50% for any column, they can be converted to 'categorical'.

In [10]:
# Number of unique values in each string column
# NB: df_loans is currently the first chunk

df_loans_string = df_loans.select_dtypes(include=['object'])

def count_unique(df):
    for col in df:
        num_unique = len(df[col].value_counts())
        print(col, num_unique, ', Ratio: ', num_unique / 3000) # 3000 is the chunksize

count_unique(df_loans_string)
# By inspecting the output below, ALL the string columns except emp_title have 
# less than 50% unique values.

id 2538 , Ratio:  0.846
term 2 , Ratio:  0.0006666666666666666
int_rate 249 , Ratio:  0.083
grade 7 , Ratio:  0.0023333333333333335
sub_grade 35 , Ratio:  0.011666666666666667
emp_title 2207 , Ratio:  0.7356666666666667
emp_length 11 , Ratio:  0.0036666666666666666
home_ownership 5 , Ratio:  0.0016666666666666668
verification_status 3 , Ratio:  0.001
issue_d 41 , Ratio:  0.013666666666666667
loan_status 2 , Ratio:  0.0006666666666666666
pymnt_plan 1 , Ratio:  0.0003333333333333333
purpose 14 , Ratio:  0.004666666666666667
title 2004 , Ratio:  0.668
zip_code 575 , Ratio:  0.19166666666666668
addr_state 49 , Ratio:  0.01633333333333333
earliest_cr_line 358 , Ratio:  0.11933333333333333
revol_util 917 , Ratio:  0.30566666666666664
initial_list_status 1 , Ratio:  0.0003333333333333333
last_pymnt_d 95 , Ratio:  0.03166666666666667
last_credit_pull_d 102 , Ratio:  0.034
application_type 1 , Ratio:  0.0003333333333333333


# Perform the actual optimization of the string columns

In [11]:
# Convert columns with less than 50% unique values to 'categorical'
# chunck size is 3000

# Reset
df_loans_iter = pd.read_csv('loans_2007.csv', chunksize=3000)
df_loans = next(df_loans_iter)
df_loans_string = df_loans.select_dtypes(include=['object'])

def to_categorical(df, num_rows):
    # num_rows is the number of rows in the chunk
    for col in df:
        if len(df[col].value_counts()) / num_rows < 0.5:
            df[col] = df[col].astype('category')

# Testing to ascertain memory reduction
print('Before optimization: ', df_loans_string.memory_usage(deep=True).sum())
to_categorical(df_loans_string, 3000)
print('After optimization: ', df_loans_string.memory_usage(deep=True).sum())

Before optimization:  4058892
After optimization:  625612


<ipython-input-11-21c671e01568>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = df[col].astype('category')


## 4. Optimizing Numeric Columns

It looks like we were able to realize some powerful memory savings by converting to the category type and converting string columns to numeric ones.

Now let's optimize the numeric columns using the `pandas.to_numeric()` function.

**Tasks**

While working with dataframe chunks:
* Identify float columns that contain missing values, and that we can convert to a more space efficient subtype.
* Identify float columns that don't contain any missing values, and that we can convert to the integer type because they represent whole numbers.
* Based on your conclusions, perform the necessary type changes across all chunks.
* Calculate the total memory footprint and compare it with the previous one.




# Numeric columns without missing values
These can potentially be converted from floats to integers

In [12]:
# float columns without missing values
df_loans_numeric = df_loans.select_dtypes(include=['float16', 'float32', 'float64'])
for col in df_loans_numeric:
    num_missing_vals = df_loans_numeric[col].isnull().sum()
    print(col,  '=> Missing values: ', num_missing_vals) # 3000 is the chunksize

# From inspecting the output below, non of the float columns have missing values
# They are therefore all candidates for conversion to integers


member_id => Missing values:  0
loan_amnt => Missing values:  0
funded_amnt => Missing values:  0
funded_amnt_inv => Missing values:  0
installment => Missing values:  0
annual_inc => Missing values:  0
dti => Missing values:  0
delinq_2yrs => Missing values:  0
inq_last_6mths => Missing values:  0
open_acc => Missing values:  0
pub_rec => Missing values:  0
revol_bal => Missing values:  0
total_acc => Missing values:  0
out_prncp => Missing values:  0
out_prncp_inv => Missing values:  0
total_pymnt => Missing values:  0
total_pymnt_inv => Missing values:  0
total_rec_prncp => Missing values:  0
total_rec_int => Missing values:  0
total_rec_late_fee => Missing values:  0
recoveries => Missing values:  0
collection_recovery_fee => Missing values:  0
last_pymnt_amnt => Missing values:  0
collections_12_mths_ex_med => Missing values:  0
policy_code => Missing values:  0
acc_now_delinq => Missing values:  0
chargeoff_within_12_mths => Missing values:  0
delinq_amnt => Missing values:  0
pu

# Perform the actual optimization of numeric columns

In [13]:
# Convert float columns without missing values to integers

# Reset
df_loans_iter = pd.read_csv('loans_2007.csv', chunksize=3000)
df_loans = next(df_loans_iter)
df_loans_numeric = df_loans.select_dtypes(include=['float16', 'float32', 'float64'])

def to_integers(df):
    for col in df:
        if is_numeric_dtype(df[col]):
            if df[col].isnull().sum() == 0:
                df[col] = df[col].astype('int')
                df[col] = pd.to_numeric(df[col], downcast='integer')

# Testing to ascertain memory reduction
print('Before optimization: ', df_loans_numeric.memory_usage(deep=True).sum())
to_integers(df_loans_numeric)
print('After optimization: ', df_loans_numeric.memory_usage(deep=True).sum())

Before optimization:  720128
After optimization:  201128


<ipython-input-13-9fe6712e3d7f>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = df[col].astype('int')
<ipython-input-13-9fe6712e3d7f>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = pd.to_numeric(df[col], downcast='integer')


# 5. Assess the footprint improvements

# Memory usage across all chunks

In [14]:
# Memory usage across all the chunks

# Reset the iterable
df_loans_iter = pd.read_csv('loans_2007.csv', chunksize=3000)

memory_used_without_optimization = 0
memory_used_with_optimization = 0

# Iterate over all the chunks
for df_loans in df_loans_iter:
    memory_used_without_optimization += df_loans.memory_usage(deep=True).sum()

    df_loans_string = df_loans.select_dtypes(include=['object'])
    df_loans_numeric = df_loans.select_dtypes(include=['float16', 'float32', 'float64'])

    # Optimize strings and numerics
    to_categorical(df_loans_string, 3000)
    to_integers(df_loans_numeric)

    memory_used_with_optimization += df_loans_string.memory_usage(deep=True).sum()
    memory_used_with_optimization += df_loans_numeric.memory_usage(deep=True).sum()

print('Total memory across chunks (before optimization): ', memory_used_without_optimization)
print('Total memory across chunks (after optimization): ', memory_used_with_optimization)

# From the output below, memory across chunks is 68.9MB

<ipython-input-11-21c671e01568>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = df[col].astype('category')
<ipython-input-13-9fe6712e3d7f>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = df[col].astype('int')


Total memory across chunks (before optimization):  68411731
Total memory across chunks (after optimization):  13548992


# 6. Automating the work above

In [15]:
# The task here is to automate the tasks in the cells above

# Determine the optimal chunk size
def optimal_chunk_size(file_name, mem_available):
    # num_rows is the initial guess for the number of rows that will yield suitably-sized chunks
    # The size of each chunk should ideally be less than 0.45 of mem_available; taking 0.45 to be on the safe side.
    num_rows = 1000

    df = pd.read_csv(file_name, nrows=num_rows)
    chunk_memory = df.memory_usage(deep=True).sum()
    # Estimate memory per row
    mem_per_row = chunk_memory / num_rows
    # Determine how many rows will achieve 0.45 x mem_available
    optimal_num_rows = int((mem_available * 0.45) / mem_per_row)
    
    # Run (uncomment) the 3 lines below if testing
    # df = pd.read_csv(file_name, nrows=optimal_num_rows)
    # Print, during dev, for testing purposes only
    # print(df.memory_usage(deep=True).sum())

    return optimal_num_rows


# The function below automates the tasks discussed in the cells above.
# IT CAN BE RUN ON ALL THE CHUNKS BY ITERATING OVER THEM APPROPRIATELY

def process_dataset(df, optimal_num_rows):
    '''
    Parameters:
    - file_name: the name of the csv file from which to read data
    - optimal_num_rows: the number of rows to give the desired chunk size
    '''   
    # 1. STRIP TRAILING '%' FROM STRING COLUMNS
    # strip_string() was defined in one of the cells above
    strip_strings(df, '%')

    # 2. CONVERT OPTIMIZATION ON STRING COLUMNS
    # to_categorical() was defined in one of the cells above
    to_categorical(df, optimal_num_rows)

    # 3. CONVERT FLOATS TO INTEGERS
    # to_integer() was defined earlier
    to_integers(df)

    return df


# Test the entire shebang!
chunk_size = optimal_chunk_size('loans_2007.csv', 10000000) # Assuming ~10MB available

# Running without automating function
df_iter = pd.read_csv('loans_2007.csv', chunksize=chunk_size)
df = next(df_iter)
print('Memory usage without optimization', df.memory_usage(deep=True).sum())

# Running with automating function
df_iter = pd.read_csv('loans_2007.csv', chunksize=chunk_size)
df = next(df_iter)
process_dataset(df, chunk_size) # This is the automation
print('Memory usage without optimization', df.memory_usage(deep=True).sum())

Memory usage without optimization 4497465
Memory usage without optimization 935785


# Conclusion

From the output of the cell above, the memory usage before optimization was 68.9Mb whereas after optimization is is 13.9MB.

It is possible that further optimizations are possible.